TODO: I will add a requirements file for this stuff

In [ ]:
! pip install smartnoise-synth

In [ ]:
! pip install git+https://github.com/ryan112358/private-pgm.git


In [ ]:
! pip install diffprivlib

In [8]:
import pandas as pd
import numpy as np
import os

### Generating Private DataFrames

#### Epsilon Regiment

In [2]:
epsilons = [(np.e ** -1, 'e^-1'), 
            (np.e ** 0, 'e^0'), 
            (np.e ** 1, 'e^1'),
            (np.e ** 2, 'e^2')]

iterations = 5

#### Load Paper DFs

In [3]:
from saw2018cross.saw2018cross import Saw2018Cross
publication = Saw2018Cross()
df_saw2018cross = publication.dataframe

Index(['sex', 'race', 'SES', 'HigherSES', 'X1STU30OCC_STEM1',
       'X2STU30OCC_STEM1', 'ninth_grade_aspirations',
       'eleventh_grade_aspirations', 'stem_career_aspirations'],
      dtype='object')


#### Iterative generate

In [4]:
# Add a json domain file for each paper here, MST requirement.
domains = {
    "saw2018cross": "domains/saw2018cross-domain.json"
}

In [5]:
from snsynth.mst import MSTSynthesizer
from snsynth.preprocessors import GeneralTransformer, BaseTransformer
from snsynth.pytorch import PytorchDPSynthesizer
from snsynth.pytorch.nn import PATECTGAN

/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/mbi/__init__.py:15: UserWarning: MixtureInference disabled, please install jax and jaxlib
  warnings.warn('MixtureInference disabled, please install jax and jaxlib')


In [6]:
df_map = {
    "saw2018cross": df_saw2018cross
}

In [10]:
for pub_name, df in df_map.items():
    print('Generating: ' + pub_name)
    if not os.path.exists('private_data/' + str(pub_name)):
        os.mkdir('private_data/' + str(pub_name))
    for (eps, str_eps) in epsilons:
        if not os.path.exists('private_data/' + str(pub_name) + '/' + str_eps):
            os.mkdir('private_data/' + str(pub_name) + '/' + str_eps)
        for it in range(iterations):
            # Folder for deposit
            folder_name = 'private_data/' + str(pub_name) + '/' + str_eps + '/'
            
            if not os.path.isfile(folder_name + 'mst_' + str(it) + '.pickle'):
                # The MST Synthesis
                mst = MSTSynthesizer(domain=pub_name, domains_dict=domains)
                mst.fit(df)
                sample_size = len(df)
                mst_synth_data = mst.sample(sample_size)
                mst_synth_data.to_pickle(folder_name + 'mst_' + str(it) + '.pickle')

            if not os.path.isfile(folder_name + 'patectgan_' + str(it) + '.pickle'):
                # The PATECTGAN Synthesis
                preprocess_factor = 0.1
                patectgan = PytorchDPSynthesizer(eps, 
                                                PATECTGAN(preprocessor_eps=(preprocess_factor * eps)), 
                                                preprocessor=None)
                patectgan.fit(
                    df,
                    categorical_columns=list(df.columns),
                    transformer=BaseTransformer,
                )
                sample_size = len(df)
                patectgan_synth_data = patectgan.sample(sample_size)
                patectgan_synth_data.to_pickle(folder_name + 'patectgan_' + str(it) + '.pickle') 

            



Generating: saw2018cross
